# 1. LP Model for L1-SVM

This is the Julia code for the MATH271 class at Carleton College in Spring 2025 term with Rob Thompson.

Final Project: Understanding Breast Cancer Diagnosis and Prognosis via Linear Programming
Authors: Palmy Klangsathorn and Angelina Kong
Date: 06/05/2025

This project explores the application of linear programming techniques to the problem of breast cancer diagnosis and prognosis. We will utilize the Wisconsin Breast Cancer Diagnostic (WBCD) dataset to build and evaluate a predictive model. Our approach will focus on formulating the classification task as a linear program, contrasting it with traditional methods, and analyzing its performance through various statistical metrics and visualizations.

In this part, we used The L1-SVM, which is a type of Support Vector Machine that we solved using linear programming. It tries to draw a line (or hyperplane) that best separates the two types of tumors. It also uses something called L1 regularization, which helps pick out only the most important features by setting some feature weights to zero. This makes the model simpler and often more accurate.

In [ ]:
# Step 1: Read the data and store it in the dictionary named 'wdbc_data'.

# this function read the dataset which is a text file and make a dictionary of the dataset
function read_to_dict(file_path::String)
  data_dict = Dict{String, Vector{Any}}()
  try
      open(file_path, "r") do io
          for line in eachline(io)
              # split each element in the line by comma
              elements = split(line, ',')
              if isempty(elements) || all(isempty, elements)
                  continue
              end

              # the first element is the key and the rest form the list
              key = String(strip(elements[1]))

              # For 'wdbc.data', the first element is an ID, the second is 'M' or 'B' (diagnosis),
              # and the rest are floating-point numbers.
              data_list = Vector{Any}()
              push!(data_list, String(strip(elements[2]))) # diagnosis 'M' or 'B'
              for i in 3:length(elements)
                  val_str = strip(elements[i])
                  if !isempty(val_str)
                      try
                          push!(data_list, parse(Float64, val_str))
                      catch
                          push!(data_list, val_str)
                      end
                  end
              end
              data_dict[key] = data_list
          end
      end
  catch e
      if isa(e, SystemError) && e.errnum == Base.UV_ENOENT
          println("Error: File not found")
      else
          rethrow(e)
      end
  end
  return data_dict
end

# our dataset
file_path = "wdbc.data"
wdbc_data = read_to_dict(file_path)
println("Number of entries in dictionary: $(length(wdbc_data))")

# ----------------------------------------------------------------
# check check
specific_key = "848406" 
if haskey(wdbc_data, specific_key)
    println("\nData for key '$specific_key':")
    println(wdbc_data[specific_key])
else
    println("\nKey '$specific_key' not found in the dictionary.")
end

# columns info
col_names = [
    "ID", "Diagnosis",
    "radius1", "texture1", "perimeter1", "area1", "smoothness1", "compactness1", "concavity1", "concave_points1", "symmetry1", "fractal_dimension1",
    "radius2", "texture2", "perimeter2", "area2", "smoothness2", "compactness2", "concavity2", "concave_points2", "symmetry2", "fractal_dimension2",
    "radius3", "texture3", "perimeter3", "area3", "smoothness3", "compactness3", "concavity3", "concave_points3", "symmetry3", "fractal_dimension3"
]

print("\nThere are  $(length(col_names)) columns")
print("\nThere are  $(length(col_names)-2) features")

In [ ]:
# Step 2: Data Prepocessing

using DataFrames

# convert all column names to Symbols, which is the standard for DataFrames
column_names = Symbol.(col_names)

row_tuples = []
for (id, values) in wdbc_data
    # create a vector for the current row's values
    current_row_values = Vector{Any}([id, values[1]]) # start with ID and Diagnosis
    append!(current_row_values, values[2:end]) # append all features

    # Create a NamedTuple for the current row
    # This maps the column names (Symbols) to their respective values
    push!(row_tuples, NamedTuple{Tuple(column_names)}(current_row_values))
end

# Create the DataFrame
df = DataFrame(row_tuples)

# Now, we can run our subsequent Julia code on 'df'

In [ ]:
# Step 3: Encode diagnosis: M = 1, B = -1

# The 'Diagnosis' column in the original data contains categorical string labels:
# "M" for malignant and "B" for benign tumors.

# Most machine learning algorithms require numeric inputs, not strings. 
# So we need to convert these string labels into numeric values.

# In this step, we use the `ifelse.` broadcasting function to perform element-wise conversion:
# - If the diagnosis is "M" (malignant), we encode it as 1.0
# - If the diagnosis is "B" (benign), we encode it as -1.0

df.Diagnosis = ifelse.(df.Diagnosis .== "M", 1, -1)

In [ ]:
# Step 4: Extract features and normalize (using StatsBase.zscore)
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()

# Pkg.add("StatsBase")
using StatsBase 
using DataFrames

# Extract features from 3rd column to end
X = Matrix(df[:, 3:end]) 
y = df.Diagnosis

X = zscore(X, 1) # Normalize columns (dims=1 means normalize each column)

# Now, X is normalized

# By applying zscore(X, 1), we transform each column (feature) of X so that it has a mean of 0 and a standard deviation of 1. 
# features in the original dataset often have different scales and units (like "radius" might be in millimeters, "area" in square millimeters,
# "smoothness" a dimensionless value). Without normalization, features with larger numerical ranges or higher magnitudes would implicitly 
# contribute more to the distance calculations and objective functions of algorithms. Normalization ensures that all features contribute 
# proportionally to the model, preventing features with larger scales from dominating the learning process

In [ ]:
# confirm sizes
println("\n--- Checking sizes before shuffleobs ---")
println("Size of X (rows, cols): ", size(X))
println("Length of y (rows): ", length(y))
println("Are X rows and y length equal? ", size(X, 1) == length(y))
println("------------------------------------")

In [ ]:
# Step 5: Train-test split 

# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
using MLJBase 
using Random 

# 'X' (Matrix{Float64}) and 'y' (Vector) are already normalized
# get the total number of observations
num_observations = size(X, 1)

# create a vector of indices
indices = collect(1:num_observations)

# shuffle the indices in-place
Random.shuffle!(indices)

# determine split point
split_point = floor(Int, num_observations * 0.8)

# split the shuffled indices into train and test sets
train_indices = indices[1:split_point]
test_indices = indices[split_point+1:end]

# use these shuffled indices to create your train and test sets
X_train = X[train_indices, :]
y_train = y[train_indices]

X_test = X[test_indices, :]
y_test = y[test_indices]


println("X_train dimensions: ", size(X_train))
println("y_train length: ", length(y_train))
println("X_test dimensions: ", size(X_test))
println("y_test length: ", length(y_test))

# 1.  LP Classifier (L1-SVM) using JuMP

We begin with an L1-regularized. We do this by using an L1-regularized Support Vector Machine (L1-SVM), which we reformulate as a linear programming (LP) problem. This LP-based approach helps us make predictions and also makes the model easier to understand, since it tends to focus only on the most relevant features.

In [ ]:
# Step 6: LP Classifier (L1-SVM) using JuMP

import Pkg
Pkg.add("GLPK")
using JuMP
using GLPK 
using LinearAlgebra 
using Statistics

# since X_train, y_train, X_test, y_test are already defined and are numerical (e.g., 1.0 and -1.0)
y_train_numeric = y_train
y_test_numeric = y_test

# Use these numeric labels for training and prediction
n, d = size(X_train) # n is now the number of training samples

# Define the penalty parameter C for the slack variables.
# This is a hyperparameter you might tune. A common starting point is 1.0.
C_penalty = 1.0

# Create the LP Model
model_lp = JuMP.Model(GLPK.Optimizer)

@variable(model_lp, w_plus[1:d] >= 0)  # w_j_plus >= 0
@variable(model_lp, w_minus[1:d] >= 0) # w_j_minus >= 0
@variable(model_lp, b)
@variable(model_lp, ξ[1:n] >= 0)

# Reconstruct 'w' from w_plus and w_minus for use in constraints
# Although not explicitly a variable, we define this as an expression for clarity
# and use it in the dot product.
# JuMP will handle this correctly when building the constraint matrix.
# For each j, w[j] = w_plus[j] - w_minus[j]

# Linear Objective is to Minimize L1-norm of w + C * sum of slack variables
@objective(model_lp, Min, sum(w_plus) + sum(w_minus) + C_penalty * sum(ξ))

# Linear Constraints
# The main SVM constraint: y_i * (dot(w, x_i) + b) >= 1 - ξ_i
# We need to explicitly expand dot(w, X_train[i, :]) using w_plus and w_minus
@constraint(model_lp, [i=1:n],
    y_train_numeric[i] * (
        sum(X_train[i, j] * (w_plus[j] - w_minus[j]) for j in 1:d) + b
    ) >= 1 - ξ[i]
)

# Optimizing
optimize!(model_lp)

# Check if the optimization was successful
if termination_status(model_lp) == MOI.OPTIMAL
    # Recover optimal w and b
    w_opt_lp = value.(w_plus) .- value.(w_minus) # w_j = w_j_plus - w_j_minus
    b_opt_lp = value(b)

    # Step 7: Predict and Evaluate for LP SVM
    function predict_lp(X, w_lp, b_lp)
        return sign.(X * w_lp .+ b_lp)
    end

    y_pred_lp = predict_lp(X_test, w_opt_lp, b_opt_lp)

    # evaluation metrics
    TP_lp = sum((y_test_numeric .== 1) .&& (y_pred_lp .== 1))
    TN_lp = sum((y_test_numeric .== -1) .&& (y_pred_lp .== -1))
    FP_lp = sum((y_test_numeric .== -1) .&& (y_pred_lp .== 1))
    FN_lp = sum((y_test_numeric .== 1) .&& (y_pred_lp .== -1))

    println("\n--- Confusion Matrix (LP SVM) ---")
    println("True Positives (TP):  ", TP_lp)
    println("True Negatives (TN):  ", TN_lp)
    println("False Positives (FP): ", FP_lp)
    println("False Negatives (FN): ", FN_lp)

    accuracy_lp = (TP_lp + TN_lp) / (TP_lp + TN_lp + FP_lp + FN_lp)
    println("\nTest accuracy (LP SVM): ", round(accuracy_lp * 100, digits=2), "%")

    precision_lp = TP_lp / (TP_lp + FP_lp)
    println("Precision (LP SVM): ", round(precision_lp * 100, digits=2), "%")

    recall_lp = TP_lp / (TP_lp + FN_lp)
    println("Recall (Sensitivity) (LP SVM): ", round(recall_lp * 100, digits=2), "%")

    f1_score_lp = 2 * (precision_lp * recall_lp) / (precision_lp + recall_lp)
    println("F1-Score (LP SVM): ", round(f1_score_lp * 100, digits=2), "%")

    specificity_lp = TN_lp / (TN_lp + FP_lp)
    println("Specificity (LP SVM): ", round(specificity_lp * 100, digits=2), "%")

else
    println("LP Optimization failed with status: ", termination_status(model_lp))
    println("Primal Status: ", primal_status(model_lp))
    println("Dual Status: ", dual_status(model_lp))
    w_opt_lp = nothing
    b_opt_lp = nothing
end


In [ ]:
import MultivariateStats
using Plots
# using MathOptInterface as MOI # Uncomment if MathOptInterface is needed 
# specifically in this cell for `termination_status`

# Check Optimization Status and Plotting Logic for Decision Boundary or PCA 
if termination_status(model_lp) == MOI.OPTIMAL
    if d == 2
        println("\n--- Generating 2D Decision Boundary Plot ---")
        x_min, x_max = extrema(X_train[:, 1])
        y_min, y_max = extrema(X_train[:, 2])

        x_buffer = (x_max - x_min) * 0.1
        y_buffer = (y_max - y_min) * 0.1
        plot_x_min, plot_x_max = x_min - x_buffer, x_max + x_buffer
        plot_y_min, plot_y_max = y_min - y_buffer, y_max + y_buffer

        num_grid_points = 100
        test_range_x = range(plot_x_min, stop=plot_x_max, length=num_grid_points)
        test_range_y = range(plot_y_min, stop=plot_y_max, length=num_grid_points)

        Z = Matrix{Float64}(undef, num_grid_points, num_grid_points)

        for (i, x_val) in enumerate(test_range_x)
            for (j, y_val) in enumerate(test_range_y)
                grid_point = [x_val y_val]
                # Ensure predict_lp handles a 1x2 matrix correctly,
                # or reshape grid_point if predict_lp expects a vector.
                Z[i, j] = predict_lp(grid_point, w_opt_lp, b_opt_lp)[1]
            end
        end

        p = plot(;
            xlim=(plot_x_min, plot_x_max),
            ylim=(plot_y_min, plot_y_max),
            aspect_ratio=1,
            title="L1-SVM Decision Boundary",
            xlabel="Feature 1",
            ylabel="Feature 2",
            legend=:outertopright,
            left_margin=5Plots.mm 
        )

        contourf!(
            p,
            test_range_x,
            test_range_y,
            Z;
            levels=1,
            color=cgrad(:redsblues),
            alpha=0.7,
            colorbar_title="Predicted Class",
            label="",
        )

        X1 = X_train[y_train_numeric .== -1, :]
        X2 = X_train[y_train_numeric .== 1, :]

        scatter!(p, X1[:, 1], X1[:, 2]; color=:red, label="Class -1", markershape=:circle, markersize=5)
        scatter!(p, X2[:, 1], X2[:, 2]; color=:blue, label="Class 1", markershape=:xcross, markersize=5)

        display(p)
        # savefig(p, "svm_decision_boundary_2d.png") # Uncomment to save

    else # d > 2, so proceed with PCA visualization
        println("\n--- Attempting 2D Visualization via PCA ---")

        # Ensure X_train is Float64 and correctly oriented (features as rows for MultivariateStats.jl)
        X_train_for_pca = permutedims(X_train) # Transpose X_train to be (features x samples)

        # Corrected fit call:
        M = MultivariateStats.fit(MultivariateStats.PCA, X_train_for_pca; maxoutdim=2)

        # Corrected transform call:
        X_train_pca = MultivariateStats.transform(M, X_train_for_pca) # This will be 2 x n_samples

        # Now, for plotting, we need (n_samples x 2)
        X_train_pca_plot = permutedims(X_train_pca)

        # 3. Determine plot limits from transformed data
        x_min_pca, x_max_pca = extrema(X_train_pca_plot[:, 1])
        y_min_pca, y_max_pca = extrema(X_train_pca_plot[:, 2])

        x_buffer_pca = (x_max_pca - x_min_pca) * 0.1
        y_buffer_pca = (y_max_pca - y_min_pca) * 0.1
        plot_x_min_pca, plot_x_max_pca = x_min_pca - x_buffer_pca, x_max_pca + x_buffer_pca
        plot_y_min_pca, plot_y_max_pca = y_min_pca - y_buffer_pca, y_max_pca + y_buffer_pca

        p_pca = plot(;
            xlim=(plot_x_min_pca, plot_x_max_pca),
            ylim=(plot_y_min_pca, plot_y_max_pca),
            aspect_ratio=1,
            title="L1-SVM Data in PCA Space (d=$(d) reduced to 2)",
            xlabel="Principal Component 1",
            ylabel="Principal Component 2",
            legend=:outertopright,
            left_margin=5Plots.mm # <-- Added this line to fix y-axis label cutoff
        )
        
        # Scatter plot for transformed training data
        X1_pca = X_train_pca_plot[y_train_numeric .== -1, :]
        X2_pca = X_train_pca_plot[y_train_numeric .== 1, :]

        scatter!(p_pca, X1_pca[:, 1], X1_pca[:, 2]; color=:red, label="Class Benign", markershape=:circle, markersize=5)
        scatter!(p_pca, X2_pca[:, 1], X2_pca[:, 2]; color=:blue, label="Class Malignant", markershape=:xcross, markersize=5)

        display(p_pca)
        savefig(p_pca, "images/svm_pca_plot.png") 
    end
else
    println("LP Optimization failed. No visualization for decision boundary or PCA was generated.")
end

In [ ]:
using Plots
using Printf 

# Plotting Feature Coefficients (runs only if optimization was successful and d > 2)
if termination_status(model_lp) == MOI.OPTIMAL && d > 2
    
    feature_names = col_names[3:end]

    println("\n--- Plotting Feature Coefficients ---")
    feature_indices = 1:d 

    annotations = []
    
    # Determine the maximum absolute coefficient for scaling
    max_abs_coeff = isempty(w_opt_lp) ? 0.0 : maximum(abs.(w_opt_lp))

    # Define annotation font size and margin
    annotation_fontsize = 5 # You can adjust this value (e.g., 6, 8, 9)
    
    # Margin for text placement. This is a small value relative to the bar height.
    # For text INSIDE, a percentage of the bar's own height (e.g., 80%) or a fixed small offset works.
    # For text OUTSIDE (for very short bars), a percentage of the max_abs_coeff works.
    outside_margin = max_abs_coeff * 0.05 # 5% of the largest absolute coefficient value for outside placement
    inside_margin_factor = 0.8 # Place text at 80% of the bar's height (from base)
    
    # Define a threshold below which text might not fit well inside the bar
    # This might need fine-tuning based on your specific plot range and font size
    min_coeff_for_inside_text = max_abs_coeff * 0.1 # If bar height is less than 10% of max, place text outside

    # If all coefficients are zero, set a small default for positioning
    if iszero(max_abs_coeff)
        outside_margin = 0.05 
        min_coeff_for_inside_text = 0.01
    end

    for i in 1:length(w_opt_lp)
        coeff_value = w_opt_lp[i]
        text_label = @sprintf "%.2f" coeff_value
        
        y_pos = 0.0 

        # Decide whether to place text inside or outside the bar
        if abs(coeff_value) > min_coeff_for_inside_text
            # Place text INSIDE the bar
            if coeff_value > 0
                y_pos = coeff_value * inside_margin_factor # 80% up from the base for positive bars
            else # coeff_value < 0
                y_pos = coeff_value * inside_margin_factor # 80% down from the base for negative bars
            end
        else
            # Place text OUTSIDE the bar (for very short or zero bars)
            if coeff_value > 0
                y_pos = coeff_value + outside_margin
            elseif coeff_value < 0
                y_pos = coeff_value - outside_margin
            else # coeff_value == 0
                y_pos = outside_margin # Place slightly above 0 for zero bars
            end
        end

        push!(annotations, (i, y_pos, Plots.text(text_label, annotation_fontsize)))
    end
        
    p_coeffs = bar(feature_indices, w_opt_lp,
                   title="L1-SVM Feature Coefficients (L1-Norm)",
                   xlabel="Feature Name",
                   ylabel="Coefficient Value",
                   xticks=(1:length(feature_names), feature_names),
                   xrotation=90,
                   legend=false,
                   size=(1200, 700),
                   bottom_margin=15Plots.mm,
                   left_margin=10Plots.mm,
                   annotations=annotations 
    )
    display(p_coeffs)
    savefig(p_coeffs, "images/svm_plane_coeffs.png")
else
    if termination_status(model_lp) != MOI.OPTIMAL
        println("LP Optimization failed. Cannot plot feature coefficients.")
    elseif d <= 2
        println("Skipping feature coefficient plot as d <= 2 (decision boundary was plotted).")
    end
end